# Data preprocessing

## Table of Contents

**Prep**
1. Load dataset
2. Simulate iris dataset with Nan

**Simple Imputations on Iris dataset** 
1. Nan check in dataframe
2. Duplicate check & immediate deletion
3. Delete features with zero or low variance
4. Option 1 - Null value deletion
5. Option 2 - Mean/Median/Mode imputation
6. Option 3 - Grouped Mean imputation
7. kNN imputation
8. Multivariate imputations are better - seeing in action
9. Multiple rounds of imputation


**Analyzing income dataset**
1. Deduplication & Nan Imputation
2. Dealing with numerical columns - Imputation & Scaling
3. Dealing with categorical variables
4. Combine all features back together - Categorical and numerical
5. Apply KNN


In [ ]:
import numpy as np
import pandas as pd
import sklearn as sk
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
print(f"numpy version = {np.__version__}")
print(f"pandas version = {pd.__version__}") 
print(f"sklearn version = {sk.__version__}")

### Load dataset

In [ ]:
# import load_iris function from datasets module
from sklearn.datasets import load_iris

def get_iris_df(concatenate_Xy=False):
    iris = load_iris()

    df = pd.DataFrame(iris.data, columns=iris.feature_names)
    y = iris.target

    print(f"Creating Iris dataframe with {df.shape} records and {y.shape} target records")

    if concatenate_Xy is True:
        df["target"] = y
        return df

    return df, y

In [ ]:
df, y = get_iris_df()

### Simulate a dataset with Nan

Setting random sepal length & width to be Nan

In [ ]:
# Generate some random index. We will set sepal length in those rows to be Nan
sepal_len_nan_row_idx = np.random.randint(low=0, high=df.shape[0], size=10)
sepal_len_nan_row_idx

In [ ]:
# Set selected sepal length to be Nan
df.iloc[sepal_len_nan_row_idx.tolist(),0] = np.nan

In [ ]:
# Check Nan
# Question: Why this does not work? Have we not set the nan already
df[df["sepal length (cm)"] == np.nan] 

In [ ]:
# Let us check the Nan in a different way
df.iloc[sepal_len_nan_row_idx.tolist()] #Indeed they are set to nan

In [ ]:
# Correct way of checking for nan

df[df["sepal length (cm)"].isnull()] #isnull and isna are equal. both work

Set some sepal width also to be Nan

In [ ]:
sepal_wid_nan_row_idx = np.random.randint(low=0, high=df.shape[0], size=10)
df.iloc[sepal_wid_nan_row_idx.tolist(),1] = np.nan
df[df["sepal width (cm)"].isnull()] #isnull and isna are equal. both work

<hr/>

## Basic EDA

### 1. Nan check in dataframe

In [ ]:
# Additional arguments do display all columns even when column count is large
df.info(verbose=True, show_counts=True)

In [ ]:
df.isna().any(axis=0) # Columns that have ALL non null values show as false

In [ ]:
# returns Series with True for rows that contain AT LEAST ONE na in columns
df.isna().any(axis=1)

In [ ]:
df.isna().sum() # Check null count in each column

In [ ]:
df.isna().any(axis=1).sum() #Total number of rows with Nan in any of four columns

### 2. Duplicate check & immediate deletion

In [ ]:
# returns a series with True False for every row
dupsSeries = df.duplicated() 
print(f"Number of duplicates = {dupsSeries.sum()}")

In [ ]:
df.drop_duplicates(inplace=True)
df.info() # check again to confirm duplicates gone

### 3. Delete features with zero or low variance

A quasi-constant feature, using a threshold of 0.1 means 90% of the values are similar

In [ ]:
from sklearn.feature_selection import VarianceThreshold

var_thr = VarianceThreshold(threshold = 0.25) #Removing both constant and quasi-constant
var_thr.fit(df)

var_thr.get_support() # we are not deleting here. we just want to get an idea

#also the threshold is generally less than 0.1

### 4. Option 1 - Null value deletion

In [ ]:
# Reload the iris dataframe and set null values as before
df, y = get_iris_df()
df.iloc[sepal_len_nan_row_idx.tolist(),0] = np.nan
df.iloc[sepal_wid_nan_row_idx.tolist(),1] = np.nan

In [ ]:
df[
    df["sepal length (cm)"].isnull() | 
    df["sepal width (cm)"].isnull()
]

Drop rows containing null value

In [ ]:
# conditionally dropping records
df = df.drop(
        df[
            df["sepal length (cm)"].isnull() | 
            df["sepal width (cm)"].isnull()
        ].index)
df.shape

In [ ]:
# The following is equivalent to drop all na rows in place
#df.dropna(how = 'any', inplace = True )

### 5. Option 2 - Mean/Median/Mode imputation

In [ ]:
# Reload the iris dataframe and set null values as before
df, y = get_iris_df()
df.iloc[sepal_len_nan_row_idx.tolist(),0] = np.nan
df.iloc[sepal_wid_nan_row_idx.tolist(),1] = np.nan
df[
    df["sepal length (cm)"].isnull() | 
    df["sepal width (cm)"].isnull()
]

In [ ]:
# Mean imputation directly from Pandas should also work
# df['sepal length (cm)'] = df['sepal length (cm)'].fillna(df['sepal length (cm)'].mean())

Mean imputation for numeric values

In [ ]:
df.mean() #Find mean of each column in dataframe

In [ ]:
from sklearn.impute import SimpleImputer

# we can use mean median or mode for imputation
imputer = SimpleImputer(missing_values=np.nan, strategy="mean") 
imputer.fit_transform(df) #did you notice the output is no longer dataframe? It is a numpy array

If the previous cell imputed data, why are there still Nans in dataframe?
Ans: Because imputation transforms are not in-place

In [ ]:
df[
    df["sepal length (cm)"].isnull() | 
    df["sepal width (cm)"].isnull()
]

In [ ]:
imputer = SimpleImputer(missing_values=np.nan, strategy="mean") 
X_imputed = imputer.fit_transform(df)
X_imputed[0:5,:]

1. How to know which row/col is imputed?
2. Why do we want to know which row/col is imputed 

In [ ]:
imputer = SimpleImputer(missing_values=np.nan, strategy="mean", add_indicator=True) 
X_imputed = imputer.fit_transform(df)
X_imputed[0:5]

In [ ]:
X_imputed[
    X_imputed[:,4] == 1 or X_imputed[:,5] == 1
]

https://stackoverflow.com/questions/10062954/valueerror-the-truth-value-of-an-array-with-more-than-one-element-is-ambiguous

In [ ]:
# This is a True false 1D array
X_imputed[:,4] == 1 

Does it make sense now why numpy wants clarity on what you plan to perform with OR between two True/False arrays

In [ ]:
np.logical_or(X_imputed[:,4] == 1, X_imputed[:,5] == 1)

In [ ]:
np.any([X_imputed[:,4] == 1, X_imputed[:,5] == 1], axis=0)

In [ ]:
# we will use logical or
X_imputed[
    np.logical_or(X_imputed[:,4] == 1, X_imputed[:,5] == 1)
]

### 6. Option 3 - Grouped Mean imputation

In [ ]:
# Reload the iris dataframe and set null values as before
df = get_iris_df(concatenate_Xy=True) #Note that now, we are including both X and y in df
df.iloc[sepal_len_nan_row_idx.tolist(),0] = np.nan
df.iloc[sepal_wid_nan_row_idx.tolist(),1] = np.nan

In [ ]:
df.mean()

In [ ]:
grp = df.groupby("target")
grp.get_group(0).mean()

In [ ]:
grp.get_group(1).mean()

In [ ]:
grp.get_group(2).mean()

In [ ]:
df["sepal length (cm)"].isna()

In [ ]:
df["sepal_length_nan_indicator"] = df["sepal length (cm)"].isna()

In [ ]:
df

In [ ]:
df["sepal_length_nan_indicator"] = df["sepal length (cm)"].isna().astype(int)

In [ ]:
df

In [ ]:
df["sepal_width_nan_indicator"] = df["sepal width (cm)"].isna().astype(int)

In [ ]:
df['sepal length (cm)'] = \
    df.groupby(['target'], sort=False)['sepal length (cm)'].apply(lambda x: x.fillna(x.mean()))

df['sepal width (cm)'] = \
    df.groupby(['target'], sort=False)['sepal width (cm)'].apply(lambda x: x.fillna(x.mean()))

In [ ]:
df

### 7. Knn Imputation

In [ ]:
# Reload the iris dataframe and set null values as before
df = get_iris_df(concatenate_Xy=True) #Note that now, we are NOT including y because KnnImputer will use y in distance calc
df.iloc[sepal_len_nan_row_idx.tolist(),0] = np.nan
df.iloc[sepal_wid_nan_row_idx.tolist(),1] = np.nan

In [ ]:
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=2, weights='uniform', metric='nan_euclidean')
X_imputed = imputer.fit_transform(df)
X_imputed[0:5]

### 8. Multivariate imputations are better - seeing in action

In [ ]:
df_dummy = pd.DataFrame()
df_dummy["Age"] = [np.nan, 35, 45, 25, 55, 40, 30]
df_dummy["MonthlySal_InLakhs"] = [1, 3, 5, 1.2, 6.5, 4, 2.7]
df_dummy

In [ ]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy="mean")
mean_imputed = imputer.fit_transform(df_dummy)
df_mean_imputed = pd.DataFrame(mean_imputed, columns=["Age", "MonthlySal_InLakhs"])

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

imputer = IterativeImputer()
iter_imputed = imputer.fit_transform(df_dummy)
df_iter_imputed = pd.DataFrame(iter_imputed, columns=["Age", "MonthlySal_InLakhs"])

In [ ]:
from sklearn.metrics import mean_squared_error

fig, axes = plt.subplots(nrows=1, ncols=2, sharex=True, figsize=(12, 6))
axes = np.reshape(axes, -1)

dfs = [df_mean_imputed, df_iter_imputed]
titles = ['Mean Imputation', 'Iterative Imputation']

for i, df in enumerate(dfs):
    # Plotting the data
    x = df.Age
    y = df.MonthlySal_InLakhs

    sns.scatterplot(df, x="Age", y="MonthlySal_InLakhs", ax=axes[i], color='green')
    
    # Fitting and plotting a linear regression line
    m, b = np.polyfit(x, y, 1)
    linreg = m*x + b
    axes[i].plot(x, linreg, color='black')
    
    # # Setting the titles and including the RMSE values
    axes[i].set_title(titles[i], fontsize=16, fontweight='bold')
    rmse = round(mean_squared_error(y, linreg, squared=False), 3)
    print((rmse))

plt.show()

### 9. Multiple rounds of imputation

This refers to the chicken and egg problem we talked in the class

1. First we could do a mean imputation (or grouped mean imputation) by also using missing_indicator= true
2. This flag ensures that a new feature is added that holds 0/1 values. 1 for rows whose Nan was imputed by mean. 0 otherwise
3. With the nasty nans out of the way, apply scaling
4. Now we could go back and run a pandas update (apply or other means) to replace all those rows with Nan again for which missing indicator feature is 1.
5. Then throw away the missing indicator feature
6. Apply a multivariate imputation like knn imputation for the Nans themselves

<hr/>

## Analyzing income dataset

### 1. Deduplication & Nan Imputation 

In [ ]:
df = pd.read_csv("data/income.csv")
df.head()

In [ ]:
df.info()

In [ ]:
df.isna().sum()

Is there no Nans?

In [ ]:
# TODO: What will you check in csv for Nan? 
df.isin([?]).sum()

In [ ]:
df.columns

Fix the code below

In [ ]:
# TODO: Fix the code below
df.workclass = df['workclass'].replace(?, np.nan)
df.occupation = df.occupation.replace(?, np.nan)
df['native-country'] = df[?].replace(?, ?) 

In [ ]:
df.isna().sum()

In [ ]:
# Visual pleasure for your eyes only 
sns.heatmap(df.isnull());

In [ ]:
nan_percent  = (df.isna().sum() * 100 / len(df)).round(2)
nan_percent

In [ ]:
df[df.duplicated()]

In [ ]:
print(f"There are {df.duplicated().sum()} duplicated values present.")
df[df.duplicated()].shape

In [ ]:
df.drop_duplicates(inplace = True)

In [ ]:
df[df.duplicated()]

### 2. Checking for numerical columns - Imputation & Scaling

In [ ]:
#Select numeric columns

numeric_cols = df.select_dtypes(include=['int', 'float']).columns.tolist()
print(numeric_cols)

In [ ]:
# describe stats can run only on numerical column. meaningless for others
df[numeric_cols].describe() 

In [ ]:
df["educational-num"].unique() #this is really a numerical looking ordinal field

In [ ]:
df2 = df[numeric_cols].loc[:, df[numeric_cols].columns != "educational-num"]
df2.head()

**Apply Standard Scaler on numerical columns**

In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_numeric_scaled = sc.fit_transform(df2)
X_numeric_scaled[0:5]

##### Exercise

1. Add code for imputing numerical columns if any

In [ ]:
# What will you pass as input? df2 or X_numeric_scaled?

### 3. Dealing with categorical variables

In [ ]:
df.dtypes

In [ ]:
df.nunique() # if the column listed takes on a fixed set of values then it is encoding candidate

In [ ]:
df.workclass.unique()

In [ ]:
df["workclass"].value_counts() # do for every candidate column & decide label, ordinal or one hot encoder

In [ ]:
#df.education.unique()
#df['marital-status'].value_counts()
#df.occupation.value_counts()

##### 3.1 Impute Categorical column with most frequent

In [ ]:
# Question: Why does this fail?

imputer = SimpleImputer(strategy="most_frequent")
workclass_imputed = imputer.fit_transform(df["workclass"])
workclass_imputed[0:5]

In [ ]:
# TODO: Fix the code below for the error you see 
imputer = SimpleImputer(strategy="most_frequent")
workclass_imputed = imputer.fit_transform(df["workclass"]) #Fix this line of code 
workclass_imputed[0:5]

In [ ]:
categorical_columns = df.select_dtypes(include=['object']).columns
df_categorical = df[categorical_columns]
df_categorical.head()

In [ ]:
categorical_imputed = imputer.fit_transform(df_categorical)
categorical_imputed[0:5]

And we forgot to add educational-num to categorical dataframe

In [ ]:
imputer = SimpleImputer(strategy="most_frequent")
edu_num_imputed = imputer.fit_transform(df["educational-num"].to_numpy().reshape(-1,1))
edu_num_imputed[0:5]

##### 3.2 Combine the categorical columns together

before we stack all together, we should separate the income field in categorical. It is the target variable 

In [ ]:
imputed_categorical_features = categorical_imputed[:,:-1]

target_var = categorical_imputed[:,-1:]

In [ ]:
X_categorical = np.hstack((edu_num_imputed, imputed_categorical_features))
X_categorical[0:5]

##### 3.3 Apply One Hot encoding 

In [ ]:
X_onehot = np.hstack( (X_categorical[:,:6], X_categorical[:,-1:]) )
X_onehot[0:5]

##### 3.4 Apply Label encoding

In [ ]:
# Gender and race columns are to be label encoded
X_label = X_categorical[:,-3:-1]
X_label[0:5]

In [ ]:
from sklearn.preprocessing import OneHotEncoder

..
..
??

##### 3.5 Apply Ordinal Encoder to institutional-num

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

..
..
??

##### 3.6 What about the target income? Is it Label or ordinal?

### 4. Combine all features back together - Categorical and numerical

Rememeber the order 

### 5. Apply KNN 

In [ ]:
# TODO: Do train and test split and do fit and predict  

**What about train test split? What about fit_transform() and transform()? ... Oops**

BIGGEST MISTAKE